<a href="https://colab.research.google.com/github/francescoperagine/EyeInTheSkyNotebooks/blob/main/EyeInTheSky_YOLO_VisDrone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install loguru==0.7.3 \
             tqdm==4.67.1 \
             typer==0.15.1 \
             ultralytics==8.3.78 \
             wandb==0.19.6 \
             ray==2.42.1 \
             matplotlib==3.10.0 \
             ultralytics "ray[tune]"

In [ ]:
# !rm -rf EyeInTheSkyNotebooks
# !git clone https://github.com/francescoperagine/EyeInTheSkyNotebooks.git

In [ ]:
# %cd EyeInTheSkyNotebooks 
# !pip install -r requirements.txt

In [ ]:
!wget https://github.com/Dao-AILab/flash-attention/releases/download/v2.7.3/flash_attn-2.7.3+cu11torch2.2cxx11abiFALSE-cp311-cp311-linux_x86_64.whl

In [ ]:
from google.colab import userdata
from loguru import logger
from ray import tune
from ultralytics import YOLO, checks, settings
import torch
import wandb
import yaml
import matplotlib.pyplot as plt

In [ ]:
class ProjectConfig:
    """Singleton class for managing project configuration and secrets."""
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    @staticmethod
    def get_config(config_file: str) -> dict:
        """Load and return configuration from YAML file."""
        with open(config_file, "r") as f:
            return yaml.safe_load(f)
        
    @staticmethod
    def get_space_dict(config) -> dict:
        """Convert config space parameters to tune.uniform objects"""
        space = {}
        for param, value in config.items():
            if isinstance(value, dict):  # It's a min/max range
                space[param] = tune.uniform(value["min"], value["max"])
            else:  # It's a discrete choice list
                space[param] = tune.choice(value)
        return space
    
    @staticmethod
    def get_device() -> str:
        try:
            return 0 if torch.cuda.is_available() else "cpu"
        except Exception as e:
            print(f"Error setting device: {e}")

In [ ]:
config = ProjectConfig.get_config("config.yaml")
device = ProjectConfig.get_device()

wandb.login(key=userdata.get("WANDB_API_KEY"))
settings.update({"wandb": True})

logger.info("Performing training for model...")
logger.info(checks())

In [ ]:
model = YOLO(f"{config['model_name']}.pt")

In [ ]:
# Prepare tune and space configurations
space = ProjectConfig.get_space_dict(config["tune"]["space"])
train_args = config["tune"]["train_args"]

In [ ]:
# Run tuning with space parameter separate
result_grid = model.tune(
    data=f"{config['dataset_name']}.yaml",
    device=device,
    space=space,
    use_ray=True,
    project_name=config["project_name"],
    **config["shared_args"],
    **train_args
)

In [ ]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

In [ ]:
for i, result in enumerate(result_grid):
    plt.plot(
        result.metrics_dataframe["training_iteration"],
        result.metrics_dataframe["mean_accuracy"],
        label=f"Trial {i}",
    )

plt.xlabel("Training Iterations")
plt.ylabel("Mean Accuracy")
plt.legend()
plt.show()